# Set up GPU
### Start setting up gpu for better performance

In [ ]:
import tensorflow as tf
import os

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Remove Dodgy Images (optional)
### Import and start removing

In [ ]:
import cv2
import imghdr
import time
from matplotlib import pyplot as plt

In [ ]:
DATA_DIR = 'data'

In [ ]:
image_exts = ['jpeg', 'jpg', 'bmp', 'png']
start_time = time.time()
for image_class in os.listdir(DATA_DIR):
    for image in os.listdir(os.path.join(DATA_DIR, image_class)):
        image_path = os.path.join(DATA_DIR, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print("Invalid Image")
                os.remove(image_path)   # Remove an image
        except Exception as e:
            print(f"Issue with image {image_path}")
end_time = time.time()   
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")

### Visualize an image

In [ ]:
# Just to visualize an image
img = cv2.imread(os.path.join(DATA_DIR, 'male', '090544.jpg.jpg'))
plt.imshow(img)

# Load Data
### Using keras utils library to load images
Will be using `image_dataset_from_directory()` from keras to load images from the `./data` directory to **data** variable

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
DATA_DIR = 'data'

In [ ]:
# Image classification by tensorflow
data = tf.keras.utils.image_dataset_from_directory(DATA_DIR, image_size=(256, 256))
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()
# Images
# print(batch[0])
# Labels
print(batch[1].shape)

### Check the class of images

In [ ]:
# 1 is male 2 is female
fix, ax = plt.subplots(ncols=4, figsize=(20, 20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

# Preprocess Data
### Start Scaling data
This step we scale the data to fit between 0-1 by dividing by 255

In [ ]:
# Scale data
data = data.map(lambda x, y: (x/255, y))

In [ ]:
scaled_it = data.as_numpy_iterator()

In [ ]:
scaled_it.next()[0].shape

### Split Data

In [ ]:
train_size = int(len(data) * .7) + 2 # 70%
val_size = int(len(data) * .2)   # 20%
test_size = int(len(data) * .1)  # 10%

In [ ]:
train_size + val_size + test_size

In [ ]:
# Initialize variables for data
train_data = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

# Deep Model
### Initilize model
Use Sequential Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
# Create a Sequential model
model = Sequential()

In [ ]:
image_width = 256
image_height = 256
channels = 3
num_classes = 1

### Model Architecture

In [ ]:
# Build the model architecture
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(image_width,image_height,channels))) # 128 x 128 by 3 channels
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(64, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

# Train the model

In [ ]:
# Log directory
LOG_DIR = 'logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR)

In [ ]:
hist = model.fit(train_data, epochs=15, validation_data=val, callbacks=[tensorboard_callback])

### Plot Performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# Evaluate Performance

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
precision = Precision()
recall = Recall()
accuracy = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    precision.update_state(y, yhat)
    recall.update_state(y, yhat)
    accuracy.update_state(y, yhat)

In [ ]:
print(f"{precision.result()}, {recall.result()}, {accuracy.result()}")

# Custom Test
Download an image an place it in the root directory

In [ ]:
# Read in an image
test1_img_dir = 'testfm.jpg'
test2_img_dir = 'testm.jpg'
img = cv2.imread(test1_img_dir)

# Resize to 256x256
resize_img = tf.image.resize(img, (256, 256))
plt.imshow(resize_img.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize_img/255, 0))
yhat

# Save the Model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models','imageclassifier.h5'))

In [105]:
new_model = load_model('./models/imageclassifier.h5')

In [106]:
new_model.predict(np.expand_dims(resize_img/255, 0))

1/1 [==============================] - 0s 47ms/step


array([[1.]], dtype=float32)